In [10]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
import math
from sklearn.metrics import classification_report

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [11]:
import os
import sys

# Simpan current directory
notebook_dir = os.getcwd()
print(f"Original directory: {notebook_dir}")

# Pindah ke parent directory (fraud_detection)
parent_dir = os.path.dirname(notebook_dir)
os.chdir(parent_dir)
print(f"Changed to: {os.getcwd()}")

# Tambahkan ke sys.path
sys.path.append(parent_dir)

# Sekarang import akan bekerja
import src.util as utils
print("Successfully imported src.util!")

# Jika perlu, bisa kembali ke notebook directory
# os.chdir(notebook_dir)

Original directory: C:\Users\Utandra\Documents\DeployCamp_Team9\fraud_detection
Changed to: C:\Users\Utandra\Documents\DeployCamp_Team9
Successfully imported src.util!


In [3]:
config = utils.load_config()

In [4]:
dataset = pd.read_csv(config['data_path'])
dataset.head()

,transaction_id,amount,merchant_type,device_type,label
0,1,46.93,travel,tablet,0
1,2,301.01,groceries,desktop,0
2,3,131.67,others,tablet,0
3,4,91.29,electronics,desktop,0
4,5,16.96,others,mobile,0


In [5]:
dataset = dataset.drop(['transaction_id'], axis=1)

In [6]:
dataset = pd.get_dummies(dataset)

In [7]:
dataset.head()

,amount,label,merchant_type_clothing,merchant_type_electronics,merchant_type_groceries,merchant_type_others,merchant_type_travel,device_type_desktop,device_type_mobile,device_type_tablet
0,46.93,0,False,False,False,False,True,False,False,True
1,301.01,0,False,False,True,False,False,True,False,False
2,131.67,0,False,False,False,True,False,False,False,True
3,91.29,0,False,True,False,False,False,True,False,False
4,16.96,0,False,False,False,True,False,False,True,False


In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(dataset.drop(['label'], axis=1), dataset['label'], test_size=0.2, stratify=dataset['label'], random_state=42)

In [25]:
!pip install xgboost

  Using cached xgboost-3.0.2-py3-none-win_amd64.whl.metadata (2.1 kB)
Using cached xgboost-3.0.2-py3-none-win_amd64.whl (150.0 MB)



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [27]:
from xgboost import XGBClassifier

In [38]:
model_xgb = XGBClassifier(random_state=42, scale_pos_weight=50)
model_xgb.fit(X_train, y_train)
pred_test = model_xgb.predict(X_test)

print(classification_report(pred_test, y_test))

              precision    recall  f1-score   support

           0       0.93      0.95      0.94       186
           1       0.00      0.00      0.00        14

    accuracy                           0.88       200
   macro avg       0.46      0.47      0.47       200
weighted avg       0.86      0.88      0.87       200



In [17]:
from sklearn.linear_model import LogisticRegression

model_lr = LogisticRegression(class_weight='balanced')
model_lr.fit(X_train, y_train)
pred_test = model_lr.predict(X_test)
print(classification_report(pred_test, y_test))

              precision    recall  f1-score   support

           0       0.54      0.95      0.69       108
           1       0.50      0.05      0.10        92

    accuracy                           0.54       200
   macro avg       0.52      0.50      0.39       200
weighted avg       0.52      0.54      0.42       200



In [18]:
from sklearn.tree import DecisionTreeClassifier

model_dt = DecisionTreeClassifier(random_state=42, class_weight='balanced')
model_dt.fit(X_train, y_train)
pred_test = model_dt.predict(X_test)
print(classification_report(pred_test, y_test))

              precision    recall  f1-score   support

           0       0.95      0.95      0.95       190
           1       0.10      0.10      0.10        10

    accuracy                           0.91       200
   macro avg       0.53      0.53      0.53       200
weighted avg       0.91      0.91      0.91       200

